In [2]:
import numpy as np

# can be 'PIN' or 'ROLLER'
class Joint(object):
    def __init__(self, id=0, x=0, y=0):
        self.id=id
        self.x=x
        self.y=y

# list of joints: dictionary {ID : {Joint}  }, 
# list of edges: tuples (A, B)    
    
class Truss(object):
    def __init__(self, edgeList=[], joints=[], fixed_joint='', roller_joint=''):
        self.edgeList = edgeList
        self.joints = joints
        self.fixed_joint = fixed_joint
        self.roller_joint = roller_joint
        
        self.jointDict = {}
        for joint in self.joints:
            self.jointDict[joint.id] = joint
        
    

In [3]:
# truss solving function:
# takes in:
# TRUSS object (list of nodes, list of edges)
# list of forces
# AppliedForces : (JointID, magnitude, direction (in y direction))
def solve_forces(appliedForces, truss):
    numAppliedForces = len(appliedForces)
    numJoints = len(truss.joints)
    numReactions = 3 # 2 (Fx, Fy) at fixed joint, 1 (Fy) at roller
    numEdges = len(truss.edgeList)
    numEquations = numAppliedForces + numEdges + numReactions
    print(numAppliedForces)
    print(numJoints)
    print(numReactions)
    print(numEquations)
    M = np.zeros((numEquations, numEquations))
    #M system of equations to solve [<applied forces> .... <x forces in joints> <y forces in joints> ]
    #F : vector of forces to solve for [<applied forces>, <forces in beams>, <reactions>]
    
    E = np.zeros((numEquations, 1)) #solution to system of equations M*F = E
    #populate rows for  E 
    for i in range(numAppliedForces):
        E[i] = appliedForces[i][1]
        M[i][i] = 1

    i = 0 
    for i in range(numJoints):
        joint = truss.joints[i]
        jointId = joint.id
        row = i*2 + numAppliedForces
        print(row)
        for j in range(len(appliedForces)):
            col = j
            if(appliedForces[j][0] == jointId):
                M[row][col] = 1 # Force:(jointId, magnitude)
        
        for j in range(len(truss.edgeList)):
            col = j+numAppliedForces
            edge = truss.edgeList[j]
            otherJointId = None
            if(edge[0] == jointId):
                otherJointId = edge[1]
            elif(edge[1] == jointId):
                otherJointId = edge[0]
            if(otherJointId != None):
                otherJoint = truss.jointDict[otherJointId]
                x1 = joint.x
                y1 = joint.y
                x2 = otherJoint.x
                y2 = otherJoint.y
                dx = x2 - x1
                dy = y2 - y1
                
                theta = np.abs(np.arctan(dy/dx))
                x_multiplier = np.sign(dx) * np.cos(theta)
                y_multiplier = np.sign(dy) * np.sin(theta)
                
                M[row][col] = y_multiplier
                M[row+1][col] = x_multiplier
        if(joint.id == truss.fixed_joint):
            M[row][numEquations - 3] = 1 # reaction, assume in positive direction
            M[row + 1][numEquations - 2] = 1 #y reaction, assume in positive direction
        if(joint.id == truss.roller_joint):
            M[row][numEquations - 1] = 1
    
    F = np.linalg.solve(M, E)
    F = np.around(F,2)
    return M, E, F

In [60]:
#joint_x_list = [0, 0.5, 1]
#joint_y_list = [0, 0.866, 0]
#jointIds = ['A', 'B', 'C']
joint_x_list = [0, 1, 2, 3, 4]
joint_y_list = [0, 1, 0, 1, 0]
jointIds = ['A', 'B', 'C', 'D', 'E']
jointList = []
for i in range(len(jointIds)):
    joint = Joint(jointIds[i],joint_x_list[i],joint_y_list[i]);
    jointList.append(joint)

edge_list = [('A', 'B'), ('B', 'C'), ('A', 'D'), ('D', 'E'), ('D', 'B'), ('E','C'), ('B', 'E')]

truss = Truss(edge_list, jointList, 'A', 'E')

applied_force_list = [('B', -100)] # 500 Newtons

In [61]:
M, E, F = solve_forces(applied_force_list, truss)

1
5
3
11
1
3
5
7
9


In [62]:
print(M)

[[ 1.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.70710678  0.          0.31622777  0.          0.          0.
   0.          1.          0.          0.        ]
 [ 0.          0.70710678  0.          0.9486833   0.          0.          0.
   0.          0.          1.          0.        ]
 [ 1.         -0.70710678 -0.70710678  0.          0.          0.          0.
  -0.31622777  0.          0.          0.        ]
 [ 0.         -0.70710678  0.70710678  0.          0.          1.          0.
   0.9486833   0.          0.          0.        ]
 [ 0.          0.          0.70710678  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.70710678  0.          0.          0.          1.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.31622777 -0.70710678  0.          0.
   0.          0.  

In [63]:
print(F)

[[-100.  ]
 [-159.1 ]
 [   0.  ]
 [ 118.59]
 [ -53.03]
 [-150.  ]
 [  -0.  ]
 [  39.53]
 [  75.  ]
 [   0.  ]
 [  25.  ]]


In [64]:
M = [
    [1,     0,      0,      0,      0,      0,      0],
    [0,     0.866,  0,      0,      0,      1,      0],
    [0,     0.5,    0,      1,      1,      0,      0],
    [-1,    -0.866, -0.866,  0,      0,      0,      0],
    [0,     -0.5,   0.5,    0,      0,      0,      0],
    [0,     0,      0.866, 0,      0,      0,      1],
    [0,     0,      -0.5,   -1,     0,      0,      0],
    ]
E = np.zeros((7, 1))
E[0] = 100

In [65]:
F = np.linalg.solve(M, E)
F = np.around(F,2)
print(F)

[[ 100.  ]
 [ -57.74]
 [ -57.74]
 [  28.87]
 [  -0.  ]
 [  50.  ]
 [  50.  ]]


In [66]:
import operator
def sort_by_x(list_of_nodes):
    list_of_nodes.sort(key = operator.attrgetter('x'))


In [67]:
def getKey(item):
    return item[1]

def calculate_distance(start_joint, end_joint):
    dx = end_joint.x - start_joint.x
    dy = end_joint.y - start_joint.y
    return np.abs(dx*dx + dy*dy)

def sorted_list_of_nodes_by_distances(list_of_joints, start_joint):
    distance_list = []
    for joint in list_of_joints:
        if(joint.id == start_joint.id):
            continue
        r = calculate_distance(start_joint, joint)
        distance_list.append((joint.id, r))
    return sorted(distance_list, key = getKey)

In [68]:
def connect_nodes(id1, id2, connected_node_table, list_of_connections):
    connected_node_table[id1].append(id2)
    connected_node_table[id2].append(id1)
    edge = (id1, id2)
    list_of_connections.append(edge)
    
def build_oonnections(list_of_joints, dict_of_symmetric_joints, center_joint_id, pin_joint_id):
    sort_by_x(list_of_joints)
    
    list_of_connections = []
    connected_node_table = {}
    #initialize table with empty lists
    for joint in list_of_joints:
        connected_node_table[joint.id] = []
    
    num_nodes = len(list_of_joints)
    num_desired_connections = 2*num_nodes - 3
    if(num_desired_connections < 3):
        return
    even_num = False
    if(num_nodes % 2 == 0):
        even_num = True
    
    index_start = 0
    index_end = index_end = int((num_nodes - 1)/2)
    if(even_num):
        index_end = int(num_nodes - 2)/2
    
    index_center_node_start = index_end
    index_center_node_end = int(index_end + 1)
    if(even_num):
        index_center_node_end = int(index_center_node_start + 2)
    
    print(index_center_node_end)
    num_connections = 0
    subset_of_nodes = list_of_joints[0:index_center_node_end]
    
    
    if(even_num): #connect 2 center joints
        joint1_id = list_of_joints[index_center_node_start].id
        joint2_id = list_of_joints[index_center_node_start+1].id
        connect_nodes(joint1_id, joint2_id, connected_node_table, list_of_connections)
    else: #connect 2 joints closest to center joint
        joint1_id = list_of_joints[index_center_node_start - 1].id
        joint2_id = dict_of_symmetric_joints[joint1_id]
        connect_nodes(joint1_id, joint2_id, connected_node_table, list_of_connections)

    for node in subset_of_nodes:
        list_of_nodes_by_distance = sorted_list_of_nodes_by_distances(subset_of_nodes, node)
        closest_node_id = list_of_nodes_by_distance[0][0]
        next_closest_node_id = list_of_nodes_by_distance[1][0]
        # check if closest node already connected
        if(closest_node_id not in connected_node_table[node.id]):
            connect_nodes(node.id, closest_node_id, connected_node_table, list_of_connections)
            #mirror the connection
            mirror_id_1 = dict_of_symmetric_joints[node.id]
            mirror_id_2 = dict_of_symmetric_joints[closest_node_id]
            connect_nodes(mirror_id_1, mirror_id_2, connected_node_table, list_of_connections)
        # check if next closest node is already connected 
        if(next_closest_node_id not in connected_node_table[node.id]):
            connect_nodes(node.id, next_closest_node_id, connected_node_table, list_of_connections)
            #mirror the connection
            mirror_id_1 = dict_of_symmetric_joints[node.id]
            mirror_id_2 = dict_of_symmetric_joints[next_closest_node_id]
            connect_nodes(mirror_id_1, mirror_id_2, connected_node_table, list_of_connections)
    
    succesful = (len(list_of_connections) == num_desired_connections) 
    return list_of_connections, connected_node_table, succesful
    

In [69]:
# test for build_connections function
#[('A', 'B'), ('B', 'C'), ('A', 'D'), ('D', 'E'), ('D', 'B'), ('E','C'), ('B', 'E')]
list_of_joints = jointList
dict_of_symmetric_joints = {'A':'E', 'B':'D', 'C':'C', 'D':'B', 'E':'A'}
center_joint_id = 'C'
pin_joint_id = 'A'
list_of_connections, connected_node_table, succesful = build_oonnections(list_of_joints, dict_of_symmetric_joints, center_joint_id, pin_joint_id)

3


In [70]:
print(succesful)
print(list_of_connections)
print(connected_node_table)

True
[('B', 'D'), ('A', 'B'), ('E', 'D'), ('A', 'C'), ('E', 'C'), ('B', 'C'), ('D', 'C')]
{'A': ['B', 'C'], 'B': ['D', 'A', 'C'], 'C': ['A', 'E', 'B', 'D'], 'D': ['B', 'E', 'C'], 'E': ['D', 'C']}
